In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 数据预处理
def parse_file():
    data = pd.read_excel("目标客户体验数据.xlsx")
    # 缺失值处理
    data["B7"]=data["B7"].replace(np.nan,0)
    data.loc[(data["B6"]==5) & (data["B7"]==0),"B7"]=1
    data.loc[(data["B6"]==6) & (data["B7"]==0),"B7"]=1
    data.loc[(data["B6"]==7) & (data["B7"]==0),"B7"]=1
    # 异常值处理
    data.drop(data[(data.B14 > data.B13) | (data.B15 > data.B13)].index, inplace=True)
    data.loc[data["a1"] > 100,"a1"] = data["a1"].mean()
    data.loc[data["a3"] > 100,"a3"] = data["a3"].mean()
    data.loc[data["a5"] > 100,"a5"] = data["a5"].mean()
    data.loc[data.B17 > 100,"B17"] = data.B17.mean()
    return data

In [3]:
# 描述性统计
def describe_statistics(data):
    # 按连续型，离散型数据更改特征顺序
    data_deal_123 = data.reindex(columns=["目标客户编号","品牌类型","a1","a2","a3","a4","a5","a6","a7","a8",
                     "B2","B4","B10","B13","B14","B15","B16","B17","B1","B3","B6",
                      "B9","B11","B12","B5","B7","B8","购买意愿"])
    # 按品牌分类
    data_deal_1 = data_deal_123[data_deal_123["品牌类型"]==1].round(2)
    data_deal_2 = data_deal_123[data_deal_123["品牌类型"]==2].round(2)
    data_deal_3 = data_deal_123[data_deal_123["品牌类型"]==3].round(2)
    return data_deal_1,data_deal_2,data_deal_3

In [4]:
# 集中趋势
def central_tendency(data):
    mean = pd.DataFrame(data.mean().round(2)).T
    median = pd.DataFrame(data.median()).T
    mode = pd.DataFrame(data.mode().loc[0,:]).T
    df = [mean,median,mode]
    data_describe = pd.concat(df)
    return data_describe

In [5]:
# 离散趋势
def dispersion(data):
    # 全距
    all_distance = data.max() - data.min()
    # 四分位差
    quantile_lst = []
    for col in data.columns:
        mean_25 = data[col].quantile(q=0.25)
        mean_75 = data[col].quantile(q=0.75)
        quantile = mean_75 - mean_25
        quantile_lst_.append(quantile)
    # 标准差    
    std = data.std()
    # 方差 
    var = data.var()
    df_all_distance = pd.DataFrame(all_distance).T
    df_std = pd.DataFrame(std).T
    df_var = pd.DataFrame(var).T
    df_quantile_lst = pd.DataFrame(quantile_lst,index=("目标客户编号","品牌类型","a1","a2","a3","a4","a5","a6","a7","a8",
                         "B2","B4","B10","B13","B14","B15","B16","B17","B1","B3","B6",
                          "B9","B11","B12","B5","B7","B8","购买意愿")).T

    df_dis = [df_all_distance,df_quantile_lst,df_std,df_var]
    data_dis_describe = pd.concat(df_dis)
    return data_dis_describe

In [6]:
# 满意度统计
def satisfaction_rate(data):
    a_list = ["a1","a2","a3","a4","a5","a6","a7","a8"]
    
    down_60 = []
    for i in a_list:
        down_60_rate = len(data[data.loc[:,i] < 60 ]) / len(data)
        down_60.append(down_60_rate)

    up_90 = []
    for i in a_list:
        up_90_rate = len(data[data.loc[:,i] >= 90 ]) / len(data)
        up_90.append(up_90_rate)

    between = []
    for i in a_list:
        between_rate = len(80 <= data[data.loc[:,i] < 90 ]) / len(data)
        between.append(between_rate)

    agree_lst = [down_60,between,up_90]
    agree_df = pd.DataFrame(agree_lst)
    return agree_df